In [87]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

In [328]:
# Read in all nights booked
nights = pd.read_csv("processed_data/boston_review.csv")
nights['date'] = pd.to_datetime(nights.date)
nights = nights[(nights.date <= datetime(2019,12,31)) & (nights.date >= datetime(2019,1,1))]
nights = nights[nights.minimum_nights_prev < 28]
nights['month'] = nights.date.dt.month
nights.head()

/Users/alanliang/repos/env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (31) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,date,name_prev,host_id_prev,host_name_prev,neighbourhood_group_prev,neighbourhood_prev,latitude_prev,longitude_prev,room_type_prev,...,adjusted_price,minimum_nights,maximum_nights,adjusted_price_prev,minimum_nights_prev,maximum_nights_prev,adjusted_price_post,minimum_nights_post,maximum_nights_post,month
2,5506,2019-02-09,**$79 Special ** Private! Minutes to center!,8229,Terry,NaN,Roxbury,42.329809,-71.095595,Entire home/apt,...,NaN,NaN,NaN,$145.00,3.0,730.0,$79.00,3.0,730.0,2
3,5506,2019-02-10,**$79 Special ** Private! Minutes to center!,8229,Terry,NaN,Roxbury,42.329809,-71.095595,Entire home/apt,...,NaN,NaN,NaN,$145.00,3.0,730.0,$79.00,3.0,730.0,2
4,5506,2019-02-11,**$79 Special ** Private! Minutes to center!,8229,Terry,NaN,Roxbury,42.329809,-71.095595,Entire home/apt,...,NaN,NaN,NaN,$145.00,3.0,730.0,$79.00,3.0,730.0,2
5,5506,2019-02-12,**$79 Special ** Private! Minutes to center!,8229,Terry,NaN,Roxbury,42.329809,-71.095595,Entire home/apt,...,NaN,NaN,NaN,$145.00,3.0,730.0,$79.00,3.0,730.0,2
6,5506,2019-02-15,**$79 Special ** Private! Minutes to center!,8229,Terry,NaN,Roxbury,42.329809,-71.095595,Entire home/apt,...,NaN,NaN,NaN,$145.00,3.0,730.0,$79.00,3.0,730.0,2


In [327]:
# Aggregate all reviews
all_reviews = pd.read_csv("boston_reviews/jan_2019_reviews.csv")
all_reviews["date"] = pd.to_datetime(all_reviews.date)
months = ["jan_2019", "feb_2019", "mar_2019", "apr_2019", "may_2019", "jun_2019", "jul_2019", "aug_2019", "sep_2019", "oct_2019", "nov_2019", "dec_2019", "jan_2020", "feb_2020"]
for m in months:
    path = "boston_reviews/"+m+"_reviews.csv"
    m_reviews = pd.read_csv(path)
    m_reviews["date"] = pd.to_datetime(m_reviews.date)
    all_reviews = all_reviews.merge(m_reviews, on = ["listing_id", "date"], how = "outer")
    all_reviews = all_reviews.groupby(["listing_id", "date"]).count().reset_index()
    all_reviews = all_reviews[all_reviews.date >= datetime(2019,1,1)]
all_reviews = all_reviews[all_reviews.date <= datetime(2019,12,31)]

# Since `nights` only contains listings with 100+ listings free per year
all_reviews = all_reviews[all_reviews.listing_id.isin(nights.id.unique())]
all_reviews['month'] = all_reviews.date.dt.month
all_reviews.head()

,listing_id,date,month
96,5506,2019-02-18,2
97,5506,2019-03-17,3
98,5506,2019-03-30,3
99,5506,2019-04-08,4
100,5506,2019-04-15,4


## Fixed Effects Analysis - by month

In [318]:
nights_by_unit_month = nights.groupby(['id', 'month']).count().reset_index()[['id', 'month', 'name_prev']].rename({'name_prev': 'nights'}, axis = 1)
reviews_by_unit_month = all_reviews.groupby(['listing_id', 'month']).count().reset_index().rename({'date': "reviews"}, axis = 1)
data_by_unit_month = nights_by_unit_month.merge(reviews_by_unit_month, 
                                                left_on = ['id', 'month'], 
                                                right_on = ['listing_id', 'month'], 
                                                how = 'left')

data_by_unit_month = data_by_unit_month.drop("listing_id", axis = 1)
data_by_unit_month = data_by_unit_month.fillna(0)
# data_by_unit_month['pre'] = (data_by_unit_month.month < 7).astype(int)
data_by_unit_month.head()

,id,month,nights,reviews
0,5506,2,7,1.0
1,5506,3,12,2.0
2,5506,4,15,2.0
3,5506,5,22,4.0
4,5506,6,23,5.0


In [322]:
cleaned_panel = data_by_unit_month.groupby(['id', 'month']).first()
cleaned_panel.head()

nights  reviews
id   month                 
5506 2           7      1.0
     3          12      2.0
     4          15      2.0
     5          22      4.0
     6          23      5.0

In [329]:
from linearmodels import PanelOLS
mod = PanelOLS(cleaned_panel.nights, cleaned_panel.reviews, entity_effects=False, time_effects=True)
res = mod.fit(cov_type='clustered', cluster_entity=True)
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 nights   R-squared:                        0.0383
Estimator:                   PanelOLS   R-squared (Between):              0.1734
No. Observations:               11925   R-squared (Within):               0.0154
Date:                Sun, Apr 05 2020   R-squared (Overall):              0.1505
Time:                        00:24:39   Log-likelihood                -4.109e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      474.21
Entities:                        1531   P-value                           0.0000
Avg Obs:                       7.7890   Distribution:                 F(1,11913)
Min Obs:                       1.0000                                           
Max Obs:                       11.000   F-statistic (robust):             211.46
                            

In [335]:
mod = PanelOLS(cleaned_panel.nights, cleaned_panel.reviews, entity_effects=True, time_effects=False)
res = mod.fit(cov_type='clustered', cluster_entity=True)
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 nights   R-squared:                        0.0155
Estimator:                   PanelOLS   R-squared (Between):              0.1589
No. Observations:               11925   R-squared (Within):               0.0155
Date:                Sun, Apr 05 2020   R-squared (Overall):              0.1382
Time:                        00:26:14   Log-likelihood                -3.926e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      163.83
Entities:                        1531   P-value                           0.0000
Avg Obs:                       7.7890   Distribution:                 F(1,10393)
Min Obs:                       1.0000                                           
Max Obs:                       11.000   F-statistic (robust):             101.49
                            

In [333]:
mod = PanelOLS(cleaned_panel.nights, cleaned_panel.reviews, entity_effects=True, time_effects=True)
res = mod.fit(cov_type='clustered', cluster_entity=True)
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 nights   R-squared:                        0.0020
Estimator:                   PanelOLS   R-squared (Between):              0.0577
No. Observations:               11925   R-squared (Within):               0.0088
Date:                Sun, Apr 05 2020   R-squared (Overall):              0.0507
Time:                        00:25:56   Log-likelihood                -3.805e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      20.760
Entities:                        1531   P-value                           0.0000
Avg Obs:                       7.7890   Distribution:                 F(1,10383)
Min Obs:                       1.0000                                           
Max Obs:                       11.000   F-statistic (robust):             12.565
                            